In [ ]:
import requests
from time import sleep

In [ ]:
# Global variables
url = "http://localhost:9999/v1"
key = "114514"

In [ ]:
import pandas as pd

df = pd.read_csv("../data/op3_per_sec.csv", index_col=0)

# drop if tick = 0
df = df[df["tick"] != 0]

# if iv > 0.4, replace with 0.4
df["iv"] = df["iv"].apply(lambda x: min(x, 0.4))
df["rv"] = df["tick"].apply(lambda x: 0.2 if x <= 150 else 0.17 if 150 < x <= 300 else 0.21 if 300 < x <= 450 else 0.24)

# is ma is nan, drop it
# if tick > 500, drop it
rtm48c = df[df["name"] == "RTM48C"]

# calulate ema
rtm48c["iv_ema_short"] = rtm48c["iv"].ewm(span=12).mean()
rtm48c["iv_ema_long"] = rtm48c["iv"].ewm(span=26).mean()


# calulate ma
rtm48c["iv_ma_short"] = rtm48c["iv"].rolling(3).mean()
rtm48c["iv_ma_long"] = rtm48c["iv"].rolling(9).mean()

# drop nan
rtm48c = rtm48c.dropna(subset=["iv_ma_short", "iv_ma_long"])

rtm48c.head(5)

In [ ]:
# set pd option to show all columns
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


1. after 500 tick, the iv is not relible anymore.
2. avoid fake signal. -> (1. use ema to replace ma. aombine rv to indicate).

| week | rv |
| --| --|
| 1     | 20% |
|2| 17%|
| 3| 21%|
|4| 24%|

In [ ]:
import seaborn as sns

# with a larger size of the figure
sns.relplot(data=df, x="tick", y="iv", hue="name", kind="line" , height=10, aspect=2)
# sns.relplot(data=df[df["name"] == "RTM52C"], x="tick", y="iv", kind="line")

# draw rv in same figure
# sns.relplot(data=df, x="tick", y="rv", kind="line", height=10, aspect=2, color="red")

In [ ]:
import matplotlib.pyplot as plt

plotted_df = rtm48c


plt.figure(figsize=(20, 10))
plt.plot(plotted_df["tick"], plotted_df["iv"], label="iv", color="blue")
               
plt.plot(plotted_df["tick"], plotted_df["iv_ema_short"], label="iv_ema_short", color="green")
plt.plot(plotted_df["tick"], plotted_df["iv_ema_long"], label="iv_ema_long", color="orange")
               
               
plt.plot(plotted_df["tick"], plotted_df["rv"], label="rv", color="purple")


plt.legend()
plt.title("IV and RV")
plt.show()

In [ ]:
import talib as ta


plotted_df = rtm48c

ta.RSI(plotted_df["iv"], timeperiod=14)
# plot rsi
plt.figure(figsize=(20, 10))
plt.plot(plotted_df["tick"], ta.RSI(plotted_df["iv"], timeperiod=14), label="rsi", color="blue")
plt.legend()
plt.title("RSI")
plt.show()


In [ ]:
macd, macdsignal, macdhist = ta.MACD(plotted_df["iv"], fastperiod=3, slowperiod=9, signalperiod=3)

# plot macd
plt.figure(figsize=(20, 10))

plt.plot(plotted_df["tick"], macd, label="macd", color="blue")
plt.plot(plotted_df["tick"], macdsignal, label="macdsignal", color="green")
plt.plot(plotted_df["tick"], macdhist, label="macdhist", color="orange")

plt.legend()
plt.title("MACD")
plt.show()


In [ ]:
# plot rtm48c with its iv, ma3 and ma10 in only one figure, three lines!
import matplotlib.pyplot as plt

rtm48c_100 = rtm48c[rtm48c["tick"] < 100]
# if ma30 > ma3, signal = 1, else signal = 0

plt.figure(figsize=(20, 10))
# ema chart
# df_merged = pd.melt(rtm48c, id_vars=['tick', 'name'], value_vars=['iv', 'iv_ema_short', 'iv_ema_long'], var_name='variable', value_name='value')
# ma chart
df_merged = pd.melt(rtm48c, id_vars=['tick', 'name'], value_vars=['iv', 'iv_ma_short', 'iv_ma_long'], var_name='variable', value_name='value')
sns.lineplot(data=df_merged, x='tick', y='value', hue='variable')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure()
rtm52c = df[df["name"] == "RTM52C"]
rtm52p = df[df["name"] == "RTM52P"]

plt.plot(rtm52c["tick"], rtm52c["iv"], label="RTM52C")
plt.plot(rtm52p["tick"], rtm52p["iv"], label="RTM52P")

plt.legend()


plt.show()